In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

Processing triggers for mime-support (3.60ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-5chaqca1
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-5chaqca1
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=485739fb50749807469d0776a41cef47077f68ebe142f90c8689a63235471055
  Stored in directory: /tmp/pip-ephem-wheel-cache-qxwypl_d/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
%ls

MyDrive/  Shareddrives/


In [ ]:
%cd MyDrive/datasets/pc_pixel_files/

/gdrive/MyDrive/datasets/pc_pixel_files


In [ ]:
%%cu

#include <bits/stdc++.h>
#include <fstream>
#include <sys/time.h>
using namespace std;

double timetaken = 0;

__global__ void make_cuda_computation (int *image, int *conv_image, int *d_rows, int *d_cols) {
		
		double filter[3][3];
		int rows, cols;
		rows = *(d_rows);
		cols = *(d_cols);
		filter[0][0] = filter[0][2] = filter[2][0] = filter[2][2] = 1.0;
		filter[0][1] = filter[1][0] = filter[1][2] = filter[2][1] = 2.0;
		filter[1][1] = 4.0;
		//filter = {{0.111111,0.111111,0.111111},{0.111111,0.111111,0.111111},{0.111111,0.111111,0.111111}};
    //filter = {{1/9,1/9,1/9},{1/9,1/9,1/9},{1/9,1/9,1/9}};
    //filter = {{1.0,2.0,1.0},{2.0,4.0,2.0},{1.0,2.0,1.0}};

		int block = blockIdx.x + blockIdx.y*blockDim.x;
		
		int id = blockIdx.x*blockDim.x+threadIdx.x;
		int i = id/cols;
		int j = id%cols;

		if (i>0 && i<rows-1 && j<cols-1 && j>0) {

			double a = double(image[(i-1)*cols + j-1])*filter[0][0] + double(image[(i-1)*cols + j])*filter[0][1] + double(image[(i-1)*cols + j+1])*filter[0][2];
			a += double(image[i*cols + j -1])*filter[1][0] + double(image[i*cols + j])*filter[1][1] + double(image[i*cols + j+1])*filter[1][2];
			a += double(image[(i+1)*cols + j-1])*filter[2][0] + double(image[(i+1)*cols + j])*filter[2][1] + double(image[(i+1)*cols + j+1])*filter[2][2];
			
			conv_image[i*cols + j] = round(a/16.0);
			
		}
}

vector<vector<int>> do_conv(vector<vector<int>> padded_data) {
    //vector<vector<int>> conv_image_data;
    vector<vector<double>> filter;

    int rows, cols;
		rows = padded_data.size();
    cols = padded_data[0].size();
		int *d_cols,*d_rows;

		int *conv_image, *d_conv_image, *image, *d_image;
		conv_image = (int *)malloc((rows)*(cols) * sizeof(int));
		image =  (int *)malloc((rows)*(cols) * sizeof(int));

		cudaMalloc(&d_conv_image, (rows)*(cols)*sizeof(int));
		cudaMalloc(&d_image, (rows)*(cols)*sizeof(int));
		cudaMalloc(&d_cols, sizeof(int));
		cudaMalloc(&d_rows, sizeof(int));

		//adding data to pointer array
		for (int i=0;i<rows;i++) {
				for (int j=0;j<cols;j++) {
						image[ i*cols + j] = padded_data[i][j]; 
						conv_image[i*cols + j] = -1;
				}
		}
		//copying host data to device data
		cudaMemcpy( d_image, image, (rows)*(cols)*sizeof(int) , cudaMemcpyHostToDevice);
		cudaMemcpy( d_cols, &cols, sizeof(int) , cudaMemcpyHostToDevice);
		cudaMemcpy( d_rows, &rows, sizeof(int) , cudaMemcpyHostToDevice);
		
		struct timeval t1, t2;

		gettimeofday(&t1, 0);
		make_cuda_computation<<<rows*cols/512, 512>>>(d_image,d_conv_image,d_rows, d_cols);
		cudaDeviceSynchronize();

		cudaMemcpy( conv_image, d_conv_image, (rows)*(cols)*sizeof(int) , cudaMemcpyDeviceToHost);
		gettimeofday(&t2, 0);

		vector<vector<int>> conv_image_data;

		int index = cols+1;
		for (int i=1;i<rows-1;i++) {
				vector<int> temp;
				for (int j=1;j<cols-1;j++) {
						temp.push_back(conv_image[index]);
						index++;
				}
				index+=2;
				conv_image_data.push_back(temp);
		}

		free(conv_image);
		cudaFree(d_conv_image);

		double time = (1000000.0*(t2.tv_sec-t1.tv_sec) + t2.tv_usec-t1.tv_usec)/1000.0;
		timetaken += time;
	  printf("Time to generate: %3.10f ms \n", time);

    return conv_image_data;

}

int main(int argc, char** argv ) {
		//taking image as input

		ifstream Myfile;
		ofstream myfile2;
		string img_str;
		vector<int> v;

		string in_filename = "2048_2048/image_data_k.txt";
		string out_filename = "2048_2048/output_k.txt";
		char c[3] = {'b','g','r'};

		for (int t=0;t<3;t++) {
			vector<vector<int>> pixel_data;
			in_filename[21] = c[t];
			Myfile.open(in_filename);
			//reading image file
			if (Myfile.is_open()) {
				while(getline(Myfile, img_str)) {
					int k=0;
					vector<int> row;
					for (int i=0;i<img_str.length();i++) {
						if (img_str[i]==' ') {
							row.push_back(k);
							k=0;
						} else if (img_str[i]=='\n') ;
						else {
							k*=10;
							k+=int(img_str[i]) - 48;
						}
					}
					pixel_data.push_back(row);
				}
			} else {
				cout<<"Error in image input";
				return 0;
			}
			Myfile.close();

			//do convolution
			vector<vector<int>> conv_data;
			conv_data = do_conv(pixel_data);


			//writing it to output.txt
			out_filename[17] = c[t];
			myfile2.open(out_filename);
			for (int i=0;i<conv_data.size();i++) {
				for (int j=0;j<conv_data[0].size();j++) {
					myfile2 << to_string(conv_data[i][j]) << " ";
				}
				myfile2 << endl;
			}
			myfile2.close();
		}
			printf("time total = %f", timetaken);
	   return 0;
}

Time to generate: 3.5130000000 ms 
Time to generate: 3.3230000000 ms 
Time to generate: 3.3350000000 ms 
time total = 10.171000
